In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_json('./trial_task.json')

##### 1. Найти тариф стоимости доставки для каждого склада

In [3]:
df['total_quantity'] = df['products'].apply(
    lambda x: sum(item['quantity'] for item in x)
)
df['tariff'] = df['highway_cost'] / df['total_quantity']

tariff = df.groupby('warehouse_name').agg(
    tariff=('tariff', 'mean')
)

print(tariff)

                     tariff
warehouse_name             
Мордор                -10.0
гиперборея            -20.0
остров невезения       -5.0
отель Лето            -25.0
хутор близ Диканьки   -15.0


##### 2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')

In [4]:
df = df.explode('products')
df[['price', 'quantity', 'product']] = df['products'].apply(pd.Series)[['price', 'quantity', 'product']]

df['income'] = df['price'] * df['quantity']
df['expenses'] = df['tariff'] * df['quantity']
df['profit'] = df['income'] + df['expenses']

grouped_df = df.groupby('product').agg(
    quantity=('quantity', 'sum'),
    income=('income', 'sum'),
    expenses=('expenses', 'sum'),
    profit=('profit', 'sum')
).reset_index()

print(grouped_df)


                    product  quantity  income  expenses   profit
0  автограф Стаса Барецкого        48   28800    -820.0  27980.0
1           билет в Израиль        58   58000   -1050.0  56950.0
2         зеленая пластинка        61     610    -920.0   -310.0
3     ломтик июльского неба        42   18900    -670.0  18230.0
4                   плюмбус        65   16250    -940.0  15310.0
5    подписка на suppi-блог        33    4950    -545.0   4405.0
6          статуэтка Ленина        68   13600    -935.0  12665.0


##### 3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

In [5]:
profit_df = df.groupby('order_id')['profit'].sum().reset_index()
profit_df.rename(columns={'profit': 'order_profit'}, inplace=True)
print(profit_df)

print("Средняя прибыль заказов:", profit_df['order_profit'].mean())

    order_id  order_profit
0        124         705.0
1       1391         490.0
2       2091        1300.0
3       2108         200.0
4       2558         355.0
..       ...           ...
95     98100        1590.0
96     98423        1170.0
97     99220        1075.0
98     99246        1755.0
99     99324        1590.0

[100 rows x 2 columns]
Средняя прибыль заказов: 1352.3


##### 4. Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

In [6]:
grouped_df = df.groupby(['warehouse_name', 'product']).agg(
    quantity=('quantity', 'sum'),
    profit=('profit', 'sum')
).reset_index()

warehouse_profit = grouped_df.groupby(
    'warehouse_name')['profit'].sum().reset_index()
df = pd.merge(
    grouped_df, warehouse_profit, on='warehouse_name', 
    suffixes=('', '_warehouse')
)
df['percent_profit_product_of_warehouse'] = (df['profit'] / df['profit_warehouse']) * 100

print(df)

         warehouse_name                   product  quantity   profit  \
0                Мордор  автограф Стаса Барецкого         4   2360.0   
1                Мордор           билет в Израиль         9   8910.0   
2                Мордор         зеленая пластинка        11      0.0   
3                Мордор     ломтик июльского неба         3   1320.0   
4                Мордор                   плюмбус         6   1440.0   
5                Мордор    подписка на suppi-блог         8   1120.0   
6                Мордор          статуэтка Ленина        10   1900.0   
7            гиперборея  автограф Стаса Барецкого        12   6960.0   
8            гиперборея           билет в Израиль        21  20580.0   
9            гиперборея         зеленая пластинка        10   -100.0   
10           гиперборея     ломтик июльского неба        13   5590.0   
11           гиперборея                   плюмбус         9   2070.0   
12           гиперборея    подписка на suppi-блог        12   15

##### 5. Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться 'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [7]:
df.sort_values(by='percent_profit_product_of_warehouse', ascending=False, inplace=True)
df['accumulated_percent_profit_product_of_warehouse'] = df.groupby('warehouse_name')['percent_profit_product_of_warehouse'].cumsum()
print(df)


         warehouse_name                   product  quantity   profit  \
21           отель Лето           билет в Израиль        15  14625.0   
8            гиперборея           билет в Израиль        21  20580.0   
1                Мордор           билет в Израиль         9   8910.0   
27  хутор близ Диканьки  автограф Стаса Барецкого        22  12870.0   
17     остров невезения     ломтик июльского неба        10   4450.0   
28  хутор близ Диканьки           билет в Израиль        10   9850.0   
19     остров невезения          статуэтка Ленина        18   3510.0   
18     остров невезения                   плюмбус        14   3430.0   
15     остров невезения           билет в Израиль         3   2985.0   
7            гиперборея  автограф Стаса Барецкого        12   6960.0   
31  хутор близ Диканьки                   плюмбус        27   6345.0   
20           отель Лето  автограф Стаса Барецкого         8   4600.0   
10           гиперборея     ломтик июльского неба        13   55

##### 6. Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A. Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

In [8]:
def category(percent):
    if percent <= 70:
        return 'A'
    elif percent <= 90:
        return 'B'
    else:
        return 'C'

df['category'] = df['accumulated_percent_profit_product_of_warehouse'].apply(category)
print(df)

         warehouse_name                   product  quantity   profit  \
21           отель Лето           билет в Израиль        15  14625.0   
8            гиперборея           билет в Израиль        21  20580.0   
1                Мордор           билет в Израиль         9   8910.0   
27  хутор близ Диканьки  автограф Стаса Барецкого        22  12870.0   
17     остров невезения     ломтик июльского неба        10   4450.0   
28  хутор близ Диканьки           билет в Израиль        10   9850.0   
19     остров невезения          статуэтка Ленина        18   3510.0   
18     остров невезения                   плюмбус        14   3430.0   
15     остров невезения           билет в Израиль         3   2985.0   
7            гиперборея  автограф Стаса Барецкого        12   6960.0   
31  хутор близ Диканьки                   плюмбус        27   6345.0   
20           отель Лето  автограф Стаса Барецкого         8   4600.0   
10           гиперборея     ломтик июльского неба        13   55